In [1]:
# import relative packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from gensim import corpora, models, similarities # corpora is formula for text and base for data processing
from cheaters import dctConstr
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
# loading carmilla.txt
with open('carmilla.txt', 'r') as f:
    corp = f.read()

dct = dctConstr(stop_words=["i", "you", "a"], ignore_case=True)
dct.constructor(corp)

# Separate paragraphs
def split_by_paragraphs(data:str) -> []:
    processed=data.lower()
    while '\n\n\n' in processed:
        processed=processed.replace('\n\n\n','\n\n')
    out = processed.split('\n\n')
    return [o.replace("\n", " ") for o in out]
# Save divided txt data to the list of pcrop
pcorp = split_by_paragraphs(corp)
# convort original txt data to sparse vectors of Bag-of-words model 
# ex. [[(13,1)]] represents a word showed once in dictinary to index of 13
pbow = [dct(para) for para in pcorp]
# print(pbow)
# count frequency of words and flip dictionary for reversal
ptfidf = [dct.tfidf(para) for para in pcorp]
pvec = [dct.bow_to_vec(p) for p in pbow]
idx_to_terms = {i:j for j, i in dct.terms.items()} # flip dictionary for reversal
print(f"The corpus consists of {len(pvec[0])} terms and {len(pcorp)} documents")

The corpus consists of 4204 terms and 676 documents


In [3]:
# data processing
# set stopwords
stoplist = set('for a of the and to in'.split())
# fliter for stopwords
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in pcorp
]
# use gensim.corpora.dictionary,.Dictionary class to assign unique integer ID number 
# for every word in the dictionary
dictionary = corpora.Dictionary(texts)
# dictionary.doc2bow(text)is to convert text to sparse vectors，[(0, 1), (1, 1)]，for example，present this word of ID(0,1)
# appears once
# and other words do not appear.
corpus = [dictionary.doc2bow(text) for text in texts]

len(dictionary)
print(list(dictionary.values())[:10])
print(list(dct.terms.keys())[:10])

2023-01-27 13:16:21,850 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2023-01-27 13:16:21,873 : INFO : built Dictionary(6062 unique tokens: ['carmilla', 'j.', 'lefanu', 'sheridan', '1872']...) from 676 documents (total 22887 corpus positions)
2023-01-27 13:16:21,874 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(6062 unique tokens: ['carmilla', 'j.', 'lefanu', 'sheridan', '1872']...) from 676 documents (total 22887 corpus positions)", 'datetime': '2023-01-27T13:16:21.874890', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 18:29:29) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


['carmilla', 'j.', 'lefanu', 'sheridan', '1872', 'prologue', '_upon', 'accompanies', 'attached', 'doctor']
['⊹', '⊰', '⊱', '∃', '∀', '⊤', '⊥', '∊', '⋃', '⋼']


In [4]:
# construct LDA num_topics为11
lda = models.ldamodel.LdaModel(corpus=pbow, id2word=idx_to_terms, num_topics=11)
# calculate perplexity
# perplexity is to mreasure if this model is fit for use, when the value of perplexity is low, the model is good for use.
perplexity = lda.log_perplexity(pbow)
print("The perplexity for the text：")
print(perplexity)

2023-01-27 13:16:27,365 : INFO : using symmetric alpha at 0.09090909090909091
2023-01-27 13:16:27,367 : INFO : using symmetric eta at 0.09090909090909091
2023-01-27 13:16:27,370 : INFO : using serial LDA version on this node
2023-01-27 13:16:27,379 : INFO : running online (single-pass) LDA training, 11 topics, 1 passes over the supplied corpus of 676 documents, updating model once every 676 documents, evaluating perplexity every 676 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-27 13:16:27,380 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2023-01-27 13:16:27,707 : INFO : -11.100 per-word bound, 2195.3 perplexity estimate based on a held-out corpus of 676 documents with 26584 words
2023-01-27 13:16:27,708 : INFO : PROGRESS: pass 0, at document #676/676
2023-01-27 13:16:27,961 : INFO : topic #1 (0.091): 0.056*"the" + 0.041*"and" + 0.033*"of" + 0.022*"to" + 0.018*"in" + 0.014

The perplexity for the text：
-7.307411620871266


In [5]:
# split data 80% training and 20% test sets
import random
random.seed(11) #set random seed
# shuffle corpus
cp = list(corpus)
random.shuffle(cp)

# split into 80% training and 20% test sets
p = int(len(cp) * .8)
cp_train = pbow[0:p]
cp_test = pbow[p:]
lda = models.ldamodel.LdaModel(corpus=cp_train, id2word=idx_to_terms, num_topics=11)
perplexity = lda.log_perplexity(cp_test)
print("perplexity for the text: ")
print(perplexity)

2023-01-27 13:16:32,041 : INFO : using symmetric alpha at 0.09090909090909091
2023-01-27 13:16:32,043 : INFO : using symmetric eta at 0.09090909090909091
2023-01-27 13:16:32,046 : INFO : using serial LDA version on this node
2023-01-27 13:16:32,052 : INFO : running online (single-pass) LDA training, 11 topics, 1 passes over the supplied corpus of 540 documents, updating model once every 540 documents, evaluating perplexity every 540 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-27 13:16:32,053 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2023-01-27 13:16:32,307 : INFO : -11.737 per-word bound, 3414.0 perplexity estimate based on a held-out corpus of 540 documents with 20241 words
2023-01-27 13:16:32,308 : INFO : PROGRESS: pass 0, at document #540/540
2023-01-27 13:16:32,506 : INFO : topic #1 (0.091): 0.030*"the" + 0.024*""" + 0.022*"of" + 0.021*"and" + 0.020*"it" + 0.018*

perplexity for the text: 
-10.81295717980642


In [ ]:
def perplexity(num_topics):
    ldamodel = models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=50)  #passes为迭代次数，次数越多越精准
    print(ldamodel.print_topics(num_topics=num_topics, num_words=7))  #num_words is the number of words under every topic.
    print(ldamodel.log_perplexity(corpus))
    return ldamodel.log_perplexity(corpus)
# Decide the best results for the number of topics by the value of perplexity
# perplecity line charts.
x = range(1,30)  #topics scale
y = [perplexity(i) for i in x]
plt.plot(x, y)
plt.xlabel('number of topics')
plt.ylabel('perplexity')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('trends by topics-perplecity')
plt.show()

2023-01-27 13:16:35,198 : INFO : using symmetric alpha at 1.0
2023-01-27 13:16:35,199 : INFO : using symmetric eta at 1.0
2023-01-27 13:16:35,201 : INFO : using serial LDA version on this node
2023-01-27 13:16:35,202 : INFO : running online (multi-pass) LDA training, 1 topics, 50 passes over the supplied corpus of 676 documents, updating model once every 676 documents, evaluating perplexity every 676 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-27 13:16:35,310 : INFO : -9.006 per-word bound, 514.1 perplexity estimate based on a held-out corpus of 676 documents with 22887 words
2023-01-27 13:16:35,310 : INFO : PROGRESS: pass 0, at document #676/676
2023-01-27 13:16:35,343 : INFO : topic #0 (1.000): 0.027*"i" + 0.014*"my" + 0.014*"was" + 0.011*"that" + 0.011*"her" + 0.010*"with" + 0.009*"she" + 0.009*"it" + 0.008*"you" + 0.008*"as"
2023-01-27 13:16:35,344 : INFO : topic diff=0.764869, rho=1.000000
2023-01-27 13:16:35,439 : INFO : -7.508 per-word bound, 182.0 

2023-01-27 13:16:37,403 : INFO : topic diff=0.000001, rho=0.235702
2023-01-27 13:16:37,500 : INFO : -7.508 per-word bound, 182.0 perplexity estimate based on a held-out corpus of 676 documents with 22887 words
2023-01-27 13:16:37,501 : INFO : PROGRESS: pass 17, at document #676/676
2023-01-27 13:16:37,534 : INFO : topic #0 (1.000): 0.027*"i" + 0.014*"my" + 0.014*"was" + 0.011*"that" + 0.011*"her" + 0.010*"with" + 0.009*"she" + 0.009*"it" + 0.008*"you" + 0.008*"had"
2023-01-27 13:16:37,535 : INFO : topic diff=0.000000, rho=0.229416
2023-01-27 13:16:37,629 : INFO : -7.508 per-word bound, 182.0 perplexity estimate based on a held-out corpus of 676 documents with 22887 words
2023-01-27 13:16:37,629 : INFO : PROGRESS: pass 18, at document #676/676
2023-01-27 13:16:37,661 : INFO : topic #0 (1.000): 0.027*"i" + 0.014*"my" + 0.014*"was" + 0.011*"that" + 0.011*"her" + 0.010*"with" + 0.009*"she" + 0.009*"it" + 0.008*"you" + 0.008*"as"
2023-01-27 13:16:37,662 : INFO : topic diff=0.000000, rho=0.2

2023-01-27 13:16:39,664 : INFO : PROGRESS: pass 34, at document #676/676
2023-01-27 13:16:39,697 : INFO : topic #0 (1.000): 0.027*"i" + 0.014*"was" + 0.014*"my" + 0.011*"that" + 0.011*"her" + 0.010*"with" + 0.009*"she" + 0.009*"it" + 0.008*"you" + 0.008*"as"
2023-01-27 13:16:39,698 : INFO : topic diff=0.000000, rho=0.166667
2023-01-27 13:16:39,794 : INFO : -7.508 per-word bound, 182.0 perplexity estimate based on a held-out corpus of 676 documents with 22887 words
2023-01-27 13:16:39,795 : INFO : PROGRESS: pass 35, at document #676/676
2023-01-27 13:16:39,828 : INFO : topic #0 (1.000): 0.027*"i" + 0.014*"was" + 0.014*"my" + 0.011*"that" + 0.011*"her" + 0.010*"with" + 0.009*"she" + 0.009*"it" + 0.008*"you" + 0.008*"had"
2023-01-27 13:16:39,829 : INFO : topic diff=0.000000, rho=0.164399
2023-01-27 13:16:39,923 : INFO : -7.508 per-word bound, 182.0 perplexity estimate based on a held-out corpus of 676 documents with 22887 words
2023-01-27 13:16:39,924 : INFO : PROGRESS: pass 36, at docume

2023-01-27 13:16:41,817 : INFO : using symmetric eta at 0.5
2023-01-27 13:16:41,818 : INFO : using serial LDA version on this node
2023-01-27 13:16:41,819 : INFO : running online (multi-pass) LDA training, 2 topics, 50 passes over the supplied corpus of 676 documents, updating model once every 676 documents, evaluating perplexity every 676 documents, iterating 50x with a convergence threshold of 0.001000


[(0, '0.027*"i" + 0.014*"was" + 0.014*"my" + 0.011*"that" + 0.011*"her" + 0.010*"with" + 0.009*"she"')]
-7.507507685320757


...

2023-01-27 13:21:44,731 : INFO : topic #19 (0.040): 0.036*"i" + 0.015*"as" + 0.015*"my" + 0.012*"had" + 0.012*"is" + 0.012*"very" + 0.010*"that" + 0.010*"you" + 0.010*"it" + 0.009*"have"
2023-01-27 13:21:44,732 : INFO : topic #21 (0.040): 0.022*"was" + 0.020*"it" + 0.016*"i" + 0.014*"had" + 0.011*"but" + 0.011*"she" + 0.010*"we" + 0.009*"you" + 0.008*"which" + 0.007*"no"
2023-01-27 13:21:44,732 : INFO : topic #9 (0.040): 0.024*"her" + 0.019*"that" + 0.018*"i" + 0.017*"she" + 0.012*"is" + 0.011*"was" + 0.010*"it" + 0.010*"my" + 0.009*"not" + 0.008*"you"
2023-01-27 13:21:44,733 : INFO : topic #7 (0.040): 0.015*"my" + 0.013*"she" + 0.013*"he" + 0.011*"his" + 0.010*"be" + 0.008*"from" + 0.008*"had" + 0.007*"been" + 0.007*"night" + 0.007*"it"
2023-01-27 13:21:44,733 : INFO : topic diff=0.140175, rho=0.353553
2023-01-27 13:21:44,874 : INFO : -8.180 per-word bound, 290.0 perplexity estimate based on a held-out corpus of 676 documents with 22887 words
2023-01-27 13:21:44,874 : INFO : PROGRESS:

2023-01-27 13:21:46,251 : INFO : PROGRESS: pass 13, at document #676/676
2023-01-27 13:21:46,334 : INFO : topic #11 (0.040): 0.037*"i" + 0.023*"my" + 0.022*"was" + 0.016*"it" + 0.012*"that" + 0.011*"you" + 0.010*"is" + 0.009*"she" + 0.009*"this" + 0.009*"with"
2023-01-27 13:21:46,335 : INFO : topic #21 (0.040): 0.022*"was" + 0.019*"it" + 0.016*"had" + 0.015*"i" + 0.011*"we" + 0.011*"but" + 0.011*"she" + 0.009*"you" + 0.008*"this" + 0.007*"no"
2023-01-27 13:21:46,336 : INFO : topic #16 (0.040): 0.027*"i" + 0.020*"that" + 0.014*"with" + 0.013*"was" + 0.013*"her" + 0.012*"my" + 0.011*"he" + 0.011*"had" + 0.010*"all" + 0.010*"as"
2023-01-27 13:21:46,336 : INFO : topic #7 (0.040): 0.015*"my" + 0.013*"she" + 0.013*"he" + 0.011*"his" + 0.011*"be" + 0.009*"from" + 0.007*"been" + 0.007*"night" + 0.007*""i" + 0.007*"had"
2023-01-27 13:21:46,337 : INFO : topic #10 (0.040): 0.022*"my" + 0.018*"i" + 0.013*"had" + 0.012*"she" + 0.012*"was" + 0.011*"as" + 0.010*"were" + 0.010*"her" + 0.010*"at" + 0.0

2023-01-27 13:21:47,708 : INFO : topic diff=0.004783, rho=0.218218
2023-01-27 13:21:47,849 : INFO : -8.147 per-word bound, 283.4 perplexity estimate based on a held-out corpus of 676 documents with 22887 words
2023-01-27 13:21:47,850 : INFO : PROGRESS: pass 20, at document #676/676
2023-01-27 13:21:47,932 : INFO : topic #24 (0.040): 0.026*"he" + 0.018*"i" + 0.016*"had" + 0.013*"my" + 0.009*"we" + 0.009*"you" + 0.008*"was" + 0.007*"which" + 0.007*"young" + 0.007*"if"
2023-01-27 13:21:47,933 : INFO : topic #13 (0.040): 0.066*"i" + 0.019*"my" + 0.017*"was" + 0.015*"you" + 0.014*"it" + 0.012*"that" + 0.012*"with" + 0.010*"not" + 0.009*"as" + 0.009*"her"
2023-01-27 13:21:47,934 : INFO : topic #3 (0.040): 0.018*"i" + 0.012*"by" + 0.012*"its" + 0.010*"as" + 0.008*"on" + 0.008*"which" + 0.007*"my" + 0.007*"but" + 0.007*"with" + 0.007*"he"
2023-01-27 13:21:47,934 : INFO : topic #9 (0.040): 0.024*"her" + 0.019*"that" + 0.018*"i" + 0.017*"she" + 0.014*"is" + 0.011*"was" + 0.010*"it" + 0.010*"my" 

2023-01-27 13:21:49,332 : INFO : topic #9 (0.040): 0.024*"her" + 0.019*"that" + 0.018*"i" + 0.018*"she" + 0.014*"is" + 0.011*"was" + 0.010*"it" + 0.010*"my" + 0.009*"not" + 0.008*"by"
2023-01-27 13:21:49,333 : INFO : topic diff=0.001886, rho=0.188982
2023-01-27 13:21:49,475 : INFO : -8.144 per-word bound, 282.8 perplexity estimate based on a held-out corpus of 676 documents with 22887 words
2023-01-27 13:21:49,476 : INFO : PROGRESS: pass 27, at document #676/676
2023-01-27 13:21:49,558 : INFO : topic #2 (0.040): 0.020*"with" + 0.018*"my" + 0.016*"her" + 0.015*"i" + 0.015*"was" + 0.014*"it" + 0.011*"had" + 0.010*"that" + 0.009*"on" + 0.009*"from"
2023-01-27 13:21:49,559 : INFO : topic #24 (0.040): 0.026*"he" + 0.018*"i" + 0.016*"had" + 0.013*"my" + 0.010*"we" + 0.008*"you" + 0.008*"was" + 0.007*"which" + 0.007*"if" + 0.007*"as"
2023-01-27 13:21:49,559 : INFO : topic #18 (0.040): 0.023*"was" + 0.021*"he" + 0.020*"that" + 0.018*"i" + 0.013*"his" + 0.013*"as" + 0.012*"her" + 0.011*"had" + 

2023-01-27 13:21:50,934 : INFO : topic #16 (0.040): 0.026*"i" + 0.020*"that" + 0.014*"with" + 0.013*"was" + 0.013*"her" + 0.012*"he" + 0.012*"my" + 0.011*"had" + 0.010*"all" + 0.010*"as"
2023-01-27 13:21:50,935 : INFO : topic #17 (0.040): 0.024*"i" + 0.018*"my" + 0.016*"you" + 0.013*"on" + 0.012*"your" + 0.012*"our" + 0.011*"was" + 0.010*"as" + 0.010*"with" + 0.008*"that"
2023-01-27 13:21:50,935 : INFO : topic diff=0.001120, rho=0.169031
